# Nuevo topic model

In [1]:
import os
import pymongo
import re
import random
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

import spacy
from spacy.lang.en import English
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords


import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models.coherencemodel import CoherenceModel

import pickle

import pyLDAvis
import pyLDAvis.gensim

nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['https', 'co', "amp", "lt", "gt"])

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:972: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:1186: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#conexion mongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["twitter-memoria"]

csv_all = mydb["csv_all"]
bd_hilos = mydb["hilos"]

In [3]:
string = '\n'

all_hilos = []
tweets = csv_all.find({})

threads1 = {}
threads2 = {}
threads3 = {}
threads4 = {}
threads5 = {}
kthreads1 = {}
kthreads2 = {}
kthreads3 = {}
kthreads4 = {}
kthreads5 = {}
hilo_Tthreads = {}
hilos_ref = []

for tweet in tweets:
    if tweet["hilo_ref"] not in hilos_ref:
        hilos_ref.append(tweet["hilo_ref"])

for hilo in hilos_ref:
    hilo_bd = bd_hilos.find_one({"hilo": hilo})
    cant_hash = len(hilo_bd["all_hashtags"])
    if cant_hash > 0 :
        tweets_hilo = csv_all.find({"hilo_ref": hilo})
        lista_aux_hilos = []
        csv = ""
        for tweet_aux in tweets_hilo:
            lista_aux_hilos.append(tweet_aux["text"])
            csv = tweet_aux["csv"]
            hilo_csv = hilo

        if csv == "csv1":
            threads1[hilo_csv] = lista_aux_hilos
            kthreads1[hilo_csv] = string.join(lista_aux_hilos)
        elif csv == "csv2":
            threads2[hilo_csv] = lista_aux_hilos
            kthreads2[hilo_csv] = string.join(lista_aux_hilos)
        elif csv == "csv3":
            threads3[hilo_csv] = lista_aux_hilos
            kthreads3[hilo_csv] = string.join(lista_aux_hilos)
        elif csv == "csv4":
            threads4[hilo_csv] = lista_aux_hilos
            kthreads4[hilo_csv] = string.join(lista_aux_hilos)
        elif csv == "csv5":
            threads5[hilo_csv] = lista_aux_hilos
            kthreads5[hilo_csv] = string.join(lista_aux_hilos)

        hilo_Tthreads[hilo_csv] = string.join(lista_aux_hilos)

                
Tthreads1 = list(kthreads1.values())
Tthreads2 = list(kthreads2.values())
Tthreads3 = list(kthreads3.values())
Tthreads4 = list(kthreads4.values())
Tthreads5 = list(kthreads5.values())

hilos_hash = Tthreads1+Tthreads2+Tthreads3+Tthreads4+Tthreads5


In [4]:
string = '\n'

threads1 = {}
threads2 = {}
threads3 = {}
threads4 = {}
threads5 = {}
kthreads1 = {}
kthreads2 = {}
kthreads3 = {}
kthreads4 = {}
kthreads5 = {}
hilo_Tthreads = {}

for hilo in hilos_ref:
    hilo_bd = bd_hilos.find_one({"hilo": hilo})
    cant_hash = len(hilo_bd["all_hashtags"])
    if cant_hash == 0 :
        tweets_hilo = csv_all.find({"hilo_ref": hilo})
        lista_aux_hilos = []
        csv = ""
        for tweet_aux in tweets_hilo:
            lista_aux_hilos.append(tweet_aux["text"])
            csv = tweet_aux["csv"]
            hilo_csv = hilo

        if csv == "csv1":
            threads1[hilo_csv] = lista_aux_hilos
            kthreads1[hilo_csv] = string.join(lista_aux_hilos)
        elif csv == "csv2":
            threads2[hilo_csv] = lista_aux_hilos
            kthreads2[hilo_csv] = string.join(lista_aux_hilos)
        elif csv == "csv3":
            threads3[hilo_csv] = lista_aux_hilos
            kthreads3[hilo_csv] = string.join(lista_aux_hilos)
        elif csv == "csv4":
            threads4[hilo_csv] = lista_aux_hilos
            kthreads4[hilo_csv] = string.join(lista_aux_hilos)
        elif csv == "csv5":
            threads5[hilo_csv] = lista_aux_hilos
            kthreads5[hilo_csv] = string.join(lista_aux_hilos)

        hilo_Tthreads[hilo_csv] = string.join(lista_aux_hilos)
        
                
Tthreads1 = list(kthreads1.values())
Tthreads2 = list(kthreads2.values())
Tthreads3 = list(kthreads3.values())
Tthreads4 = list(kthreads4.values())
Tthreads5 = list(kthreads5.values())

hilos_no_hashs = Tthreads1+Tthreads2+Tthreads3+Tthreads4+Tthreads5


In [5]:

hilos_hash = [re.sub(r"@\S+", "", sent) for sent in hilos_hash]

hilos_hash = [re.sub(r"http\S+", "", sent) for sent in hilos_hash]

#hilos_hash = [re.sub(r"#\S+", "", sent) for sent in hilos_hash]

hilos_hash = [" ".join(sent.split()) for sent in hilos_hash]


hilos_no_hashs = [re.sub(r"@\S+", "", sent) for sent in hilos_no_hashs]

hilos_no_hashs = [re.sub(r"http\S+", "", sent) for sent in hilos_no_hashs]

#hilos_no_hashs = [re.sub(r"#\S+", "", sent) for sent in hilos_no_hashs]

hilos_no_hashs = [" ".join(sent.split()) for sent in hilos_no_hashs]


In [6]:
data = hilos_hash
data[1]

'#QAnon #GreatAwakening #TheStorm #OIGReport #internetbillofrights #InternetBillOfRightsNow (4) With censuring the Social Media and with controlling foreign countries through tech. #QAnon #GreatAwakening #TheStorm #OIGReport #internetbillofrights #InternetBillOfRightsNow (3) This opeation "LIVE" by Q, in China, had something to do with him - but was not his capture. #QAnon #GreatAwakening #TheStorm #OIGReport #internetbillofrights #InternetBillOfRightsNow (2) In Clowns Operations, and #MockingbirdMedia narrative drops. #QAnon #GreatAwakening #TheStorm #OIGReport #internetbillofrights #InternetBillOfRightsNow (5) We had this "waldo" moment, in Hong Kong. #QAnon #GreatAwakening #TheStorm #OIGReport #internetbillofrights #InternetBillOfRightsNow (6) Well, what the future holds for him? Doesn\'t look too good. /END #QAnon #GreatAwakening #TheStorm #OIGReport #internetbillofrights #InternetBillOfRightsNow (1) A thread about Snow White, and his many influences: FIRST, in the Q comms.'

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [9]:
# See trigram example
for datito in data_words:
    print(trigram_mod[bigram_mod[datito]])

['create', 'path', 'to', 'green', 'card', 'for', 'investors', 'include', 'any', 'children', 'brought', 'here', 'before', 'age', 'let', 'really', 'make', 'year', 'law', 'provisional', 'green', 'cards', 'direct', 'dhs', 'to', 'allow', 'people', 'to', 'take', 'the', 'steps', 'to', 'correct', 'their', 'immigration', 'status', 'penalties', 'besides', 'deportation', 'vote', 'out', 'elected', 'officials', 'with', 'close', 'ties', 'to', 'nativist', 'white', 'nationalist', 'or', 'fearmongering', 'groups', 'such', 'as', 'fair', 'cis', 'numbersusa', 'us', 'inc', 'kkk', 'vdare', 'etc', 'give', 'immig', 'judges', 'true', 'independence', 'more', 'support', 'staff', 'backlog', 'relief', 'for', 'india', 'china', 'philippines', 'mexico', 'expand', 'esl', 'instruction', 'create', 'state', 'offices', 'of', 'new', 'americans', 'make', 'real', 'use', 'of', 'visas', 'to', 'take', 'down', 'cartels', 'make', 'filing', 'for', 'citizenship', 'free', 'or', 'almost', 'free', 'create', 'path', 'to', 'green', 'card

['pruitt', 'is', 'danger', 'to', 'the', 'public', 'yet', 'is', 'losing', 'in', 'the', 'courts', 'while', 'conning', 'into', 'saving', 'his', 'job', 'all', 'while', 'defying', 'amp', 'insulting', 'his', 'boss', 'perhaps', 'trump', 'is', 'the', 'empty', 'vessel', 'scott', 'pruitt', 'said', 'he', 'is', 'that', 'would', 'explain', 'how', 'pruitt', 'so', 'easily', 'conned', 'him', 'bootpruitt_reality_check', 'while', 'pruitt', 'is', 'dangerous', 'threat', 'to', 'the', 'health', 'of', 'our', 'families', 'his', 'sloppy', 'and', 'careless', 'legal', 'work', 'is', 'undermining', 'his', 'ability', 'to', 'even', 'implement', 'his', 'extreme', 'pro', 'polluter', 'agenda', 'in', 'several', 'key', 'cases', 'bootpruitt_reality_check', 'trump', 'is', 'also', 'defending', 'the', 'guy', 'who', 'called', 'him', 'an', 'empty', 'vessel', 'bootpruitt_reality_check', 'pruitt', 'has', 'spent', 'million', 'on', 'security', 'and', 'travel', 'in', 'just', 'over', 'year', 'his', 'security', 'detail', 'is', 'three

['the', 'real', 'issue', 'is', 'if', 'the', 'cia', 'was', 'doing', 'these', 'behavior', 'and', 'drug', 'experiments', 'in', 'the', 'wtf', 'makes', 'us', 'think', 'they', 'just', 'decided', 'to', 'stop', 'they', 'think', 'we', 'are', 'sheep', 'they', 'are', 'wrong', 'gave', 'us', 'back', 'our', 'voice', 'qanon', 'mkultra', 'redpill', 'stated', 'by', 'senator', 'kennedy', 'there', 'are', 'perhaps', 'any', 'number', 'of', 'americans', 'who', 'are', 'walking', 'around', 'today', 'who', 'were', 'given', 'drugs', 'with', 'all', 'the', 'kinds', 'of', 'physical', 'and', 'psychological', 'damage', 'than', 'can', 'be', 'caused', 'page', 'mkultra', 'qanon', 'corruption', 'part', 'of', 'mkultra', 'that', 'led', 'to', 'its', 'exposure', 'was', 'the', 'death', 'of', 'frank', 'olsen', 'who', 'was', 'drugged', 'with', 'lsd', 'by', 'the', 'cia', 'without', 'his', 'consent', 'in', 'the', 'one', 'week', 'later', 'he', 'threw', 'himself', 'off', 'balcony', 'or', 'did', 'he', 'qanon', 'cia', 'can', 'someon

['an', 'anon', 'responding', 'to', 'question', 'who', 'performs', 'in', 'circus', 'thought', 'the', 'answer', 'was', 'clowns', 'mentioned', 'snowden', 'girlfriend', 'asks', 'who', 'the', 'father', 'is', 'his', 'name', 'is', 'jonathan', 'mills', 'need', 'to', 'dig', 'into', 'his', 'background', 'appears', 'to', 'want', 'us', 'to', 'push', 'this', 'hash', 'tag', 'responds', 'to', 'an', 'anon', 'who', 'had', 'concerns', 'about', 'ibor', 'clarifies', 'st', 'amendment', 'can', 'be', 'enforced', 'upon', 'private_companies', 'yes', 'they', 'risk', 'losing', 'cash', 'but', 'clowns', 'make', 'up', 'for', 'it', 'our', 'tax', 'payer', 'dollars', 'there', 'must', 'be', 'way', 'to', 'prove', 'that', 'if', 'we', 'are', 'funding', 'we', 'should', 'control', 'just', 'theory', 'here', 'but', 'if', 'snowden', 'has', 'flipped', 'because', 'he', 'is', 'now', 'according', 'to', 'liability', 'to', 'clowns', 'like', 'jp', 'barlow', 'who', 'was', 'ed', 'became', 'is', 'team', 'china', 'assistance', 'helping',

['in', 'the', 'movie', 'the', 'ussr', 'designed', 'submarine', 'that', 'could', 'run', 'so', 'silent', 'that', 'it', 'couldn', 'be', 'detected', 'the', 'sub', 'was', 'designed', 'to', 'park', 'off', 'the', 'shore', 'of', 'america', 'to', 'deliver', 'group', 'of', 'nuclear', 'missiles', 'within', 'minutes', 'of', 'launch', 'how', 'does', 'that', 'mirror', 'deepstate', 'deepstate', 'is', 'designed', 'to', 'put', 'the', 'proper', 'people', 'in', 'place', 'to', 'be', 'able', 'to', 'subvert', 'laws', 'in', 'it', 'erosion', 'of', 'the', 'constitutional', 'form', 'of', 'government', 'we', 'now', 'live', 'under', 'deep_state', 'has', 'been', 'placing', 'these', 'people', 'for', 'decades', 'they', 'now', 'control', 'enough', 'of', 'the', 'government', 'to', 'make', 'their', 'move', 'back', 'to', 'red_october', 'one', 'of', 'the', 'final', 'threats', 'to', 'ramius', 'handing', 'the', 'russian', 'sub', 'over', 'to', 'the', 'americans', 'is', 'that', 'there', 'cook', 'who', 'actually', 'kgb', 'age

...

['for', 'those', 'who', 'are', 'lucky', 'and', 'only', 'have', 'to', 'deal', 'with', 'loss', 'of', 'rights', 'the', 'story', 'won', 'end', 'there', 'either', 'though', 'we', 'will', 'be', 'placed', 'on', 'special', 'register', 'only', 'for', 'eu', 'citizens', 'landlords', 'banks', 'employers', 'etc', 'will', 'be', 'required', 'to', 'check', 'our', 'status', 'against', 'that', 'with', 'special', 'number', 'so', 'in', 'sum', 'we', 'are', 'looking', 'at', 'very', 'large', 'group', 'of', 'people', 'people', 'who', 'have', 'contributed', 'to', 'this', 'country', 'many', 'for', 'decades', 'they', 'are', 'suddenly', 'being', 'asked', 'to', 'prove', 'their', 'status', 'in', 'an', 'environment', 'that', 'is', 'hostile', 'and', 'where', 'the', 'onus', 'is', 'on', 'the', 'person', 'under', 'investigation', 'eu', 'citizen', 'wants', 'to', 'rent', 'flat', 'there', 'other', 'person', 'who', 'also', 'wants', 'to', 'not', 'an', 'eu', 'citizen', 'the', 'eu', 'citizen', 'will', 'have', 'to', 'give', 'la

['bref', 'le', 'memo_nunes', 'est', 'resume', 'des', 'faits', 'alternatifs', 'entretenus', 'par', 'foxnews', 'et', 'alt', 'right', 'en', 'defense', 'de', 'trump', 'qui', 'sont', 'rendus', 'ici', 'honorables', 'mais', 'enjeu', 'est', 'pas', 'dans', 'les', 'partisans', 'de', 'trump', 'qui', 'sont', 'convaincus', 'priori', 'de', 'la', 'conspiration', 'du', 'systeme', 'contre', 'lui', 'addendum', 'podcast', 'de', 'mon', 'passage', 'sur', 'le', 'memo_nunes', 'et', 'trump', 'dans', 'le', 'journal', 'de', 'de', 'mathieu', 'laurent', 'sur', 'addendum', 'durant', 'la', 'soiree', 'infos', 'sont', 'venus', 'discrediter', 'le', 'memo_nunes', 'qui', 'pourrait', 'se', 'transformer', 'en', 'boomerang', 'pour', 'trump', 'part', 'sur', 'fox', 'meme', 'nunes', 'admis', 'ne', 'pas', 'avoir', 'vu', 'lui', 'meme', 'les', 'demandes', 'autorisation', 'ecoute', 'qu_il', 'critique', 'addendum', 'autre', 'part', 'breaking', 'du', 'la', 'cour', 'fisa', 'qui', 'delivra', 'autorisation', 'ecoute', 'sur', 'page', '

['robert_mercer', 'the', 'investor', 'in', 'he', 'studied', 'at', 'university', 'of', 'illinois', 'and', 'was', 'co', 'ceo', 'of', 'renaissance', 'technologies', 'an', 'old', 'us', 'hedge', 'fund', 'company', 'also', 'studied', 'amp', 'from', 'illinois', 'and', 'when', 'the', 'stock', 'mkt', 'shoot', 'up', 'suddenly', 'pull', 'out', 'all', 'investment', 'which', 'will', 'crash', 'indian', 'stock', 'market', 'was', 'this', 'planned', 'by', 'schedule', 'time', 'was', 'june', 'july', 'but', 'seems', 'it', 'is', 'now', 'postponed', 'after', 'october', 'and', 'before', 'december', 'my', 'guess', 'would', 'be', 'diwali', 'alexander_nix', 'met', 'and', 'others', 'during', 'india', 'visit', 'nix', 'is', 'well', 'connected', 'with', 'robert_mercer', 'the', 'investor', 'in', 'and', 'ca', 'is', 'scl', 'group', 'company', 'robert', 'is', 'known', 'well', 'in', 'political', 'circle', 'and', 'the', 'bomb', 'son', 'salil', 'pitroda', 'worked', 'with', 'facebook', 'salil', 'son', 'of', 'who', 'is', 'o

['ninjago', 'is', 'mixed', 'bag', 'but', 'not', 'gonna', 'say', 'this', 'isn', 'awesome', 'she', 'conked', 'out', 'hours', 'ago', 'but', 'this', 'is', 'how', 'think', 'she', 'rank', 'today', 'attractions', 'safari', 'trek', 'merry', 'go', 'round', 'miniland', 'new', 'orleans', 'playing', 'with', 'some', 'legos', 'fairy', 'tale', 'brook', 'meeting', 'nya', 'electric', 'hand', 'dryer', 'in', 'bathroom', 'tie', 'punted', 'on', 'benihana', 'she', 'too', 'tired', 'still', 'managed', 'to', 'rustle', 'up', 'dece', 'rite', 'of', 'passage', 'for', 'her', 'lifeontop', 'she', 'insisted', 'take', 'this', 'pic', 'weird', 'ugh', 'she', 'spilled', 'chocolate', 'milk', 'on', 'herself', 'we', 'cleaned', 'it', 'before', 'it', 'could', 'stain', 'but', 'we', 'have', 'less', 'total', 'chocolate', 'milk', 'now', 'we', 've', 'agreed', 'she', 'gets', 'two', 'small', 'lego', 'sets', 'on', 'this', 'trip', 'she', 'turns', 'five', 'later', 'this', 'month', 'her', 'first', 'selection', 'was', 'from', 'the', 'lego'

['to', 'my', 'fellow', 'healthcare', 'providers', 'remember', 'pain', 'is', 'an', 'important', 'vital', 'sign', 'that', 'should', 'always', 'be', 'documented', 'qanon', 'background', 'info', 'on', 'bruce', 'and', 'nellie', 'orh', 'and', 'their', 'ham', 'radio', 'license', 'qanon', 'another', 'anon', 'made', 'this', 'observation', 'qanon', 'qanon', 'today', 'due', 'to', 'speculation', 'that', 'qanon', 'is', 'not', 'the', 'original', 'who', 'posted', 'in', 'october', 'and', 'november', 'he', 'posted', 'two', 'pictures', 'that', 'seem', 'to', 'be', 'taken', 'just', 'seconds', 'apart', 'the', 'one', 'on', 'the', 'left', 'is', 'the', 'original', 'image', 'posted', 'in', 'november', 'the', 'one', 'on', 'the', 'right', 'has', 'the', 'filename', 'a_', 'png', 'info', 'on', 'ed', 'callaghan', 'trust', 'callaghan', 'qanon', 'qanon', 'asked', 'if', 'we', 've_seen', 'the', 'interview', 'with', 'paul', 'ryan', 'assume', 'he', 'speaking', 'of', 'this', 'one', 'where', 'he', 'admits', 'that', 'preside

In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [12]:
for datlem in data_lemmatized:
    print(datlem)

['create', 'path', 'green', 'card', 'investor', 'include', 'child', 'bring', 'age', 'let', 'really', 'make', 'year', 'law', 'provisional', 'green', 'card', 'direct', 'dhs', 'allow', 'people', 'take', 'step', 'correct', 'immigration', 'status', 'penalty', 'deportation', 'vote', 'elect', 'official', 'close', 'tie', 'nativist', 'white', 'nationalist', 'fearmonger', 'group', 'fair', 'cis', 'numbersusa', 'inc', 'kkk', 'vdare', 'give', 'immig', 'judge', 'true', 'independence', 'support', 'staff', 'backlog', 'relief', 'india', 'china', 'philippine', 'mexico', 'expand', 'esl', 'instruction', 'create', 'state', 'office', 'new', 'american', 'make', 'real', 'use', 'visa', 'take', 'cartel', 'make', 'file', 'citizenship', 'free', 'almost', 'free', 'create', 'path', 'green', 'card', 'longterm', 'tps', 'holder', 'recognize', 'deporte', 'group', 'asylum', 'much', 'do', 'stuck', 'border', 'wall', 'daca', 'gutt', 'protection', 'currently', 'undocumented', 'gutt', 'decade', 'work', 'pay', 'someone', 'reb

['point', 'interesting', 'time', 'misspell', 'ny', 'schneiderman', 'resign', 'left', 'message', 'learn', 'play', 'game', 'read', 'map', 'start', 'dig', 'anon', 'qanon', 'greatawaken', 'shinethelight', 'thank', 'alibaba', 'world', 'large', 'online', 'retailer', 'locate', 'china', 'alibaba', 'clinton_foundation', 'connection', 'qanon', 'shinethelight', 'pedogate', 'pizzagate', 'wwg_wga', 'itshappening', 'greatawakening', 'therainmaker', 'iran', 'metgala', 'alibaba', 'ever', 'sell', 'vial', 'adrenochrome', 'bill', 'connection', 'alibaba', 'site', 'page', 'list', 'chidlren', 'adrenochrome', 'qanon_chan', 'qanon', 'wwg_wga', 'greatawaken', 'schneiderman', 'shinethelight', 'foundation', 'misspelling', 'chidlren', 'say', 'alibaba', 'florida', 'hospital', 'foundation', 'moyer', 'foundation', 'involve', 'pedogate', 'pizzagate', 'coincidence', 'misspell', 'sort', 'mark', 'flag', 'participant', 'investigate', 'qanon', 'anon', 'already', 'know', 'trump', 'bad', 'speller', 'coincidence', 'look', 'c

['thread', 'hard', 'keep', 'threat', 'face', 'young', 'child', 'day', 'recent', 'change', 'epa', 'particularly', 'alarm', 'child', 'health', 'epa', 'announce', 'would', 'close', 'important', 'office', 'study', 'effect', 'chemical', 'exposure', 'could', 'leave', 'child', 'danger', 'expose', 'toxic', 'substance', 'epa', 'allow', 'corporation', 'release', 'pollutant', 'air', 'poor', 'air', 'quality', 'damage', 'range', 'asthma', 'lung', 'disease', 'child', 'deserve', 'environmental', 'policy', 'protect', 'health', 'protect', 'large', 'corporation', 'chemical', 'industry', 'instead', 'bootpruitt', 'recent', 'environmental', 'safety', 'change', 'threaten', 'child', 'health', 'epa', 'weaken', 'standard', 'toxic', 'metal', 'lead', 'mercury', 'air', 'water', 'child', 'particularly', 'low', 'income', 'child', 'color', 'could', 'risk', 'ingest', 'metal', 'water', 'drink', 'air', 'breathe', 'pruitt', 'reverse', 'ban', 'chlorpyrifos', 'common', 'pesticide', 'know', 'neurotoxin', 'show', 'brain', '

['time', 'say', 'tighten', 'hold', 'attractive', 'kobe', 'bryant', 'rapist', 'kobe', 'bryant', 'oscar', 'timesup', 'metoo', 'btw', 'kobebryant', 'also', 'homophobic', 'fin', 'call', 'referee', 'guess', 'okay', 'oscar', 'timesup', 'hypocrite', 'evidence', 'include', 'shirt', 'stain', 'year', 'old', 'blood', 'kobebryant', 'make', 'stay', 'clean', 'could', 'leave', 'oscar', 'oscar', 'timesup', 'pay', 'thought', 'payoff', 'always', 'mean', 'guilty', 'kobebryant', 'oscar', 'kobebryant', 'lawyer', 'use', 'victim', 'name', 'court', 'fight', 'release', 'slutsham', 'give', 'oscar', 'oscar', 'metoo', 'oscar', 'kobebryant', 'say', 'bruise', 'neck', 'happen', 'suggest', 'ask', 'michelle', 'wife', 'go', 'tell', 'anybody', 'right', 'say', 'rap', 'oscar', 'oscar', 'oscar', 'initially', 'lie', 'police', 'kobebryant', 'admit', 'sex', 'last', 'min', 'grab', 'throat', 'ask', 'jerk', 'face', 'finish', 'move', 'metoo', 'oscar', 'happen', 'kobebryant', 'lawyer', 'drag', 'victim', 'mud', 'use', 'sexual', 'hi

['link', 'column', 'deliberately', 'repeatedly', 'attack', 'character', 'accuser', 'many', 'demoralize', 'put', 'seacrest', 'red', 'carpet', 'respect', 'anyone', 'talk', 'today', 'decline', 'ryan', 'seacrest', 'many', 'platform', 'defend', 'give', 'side', 'incl', 'variety', 'say', 'ask', 'money', 'story', 'note', 'seacrest', 'rep', 'provide', 'evidence', 'variety', 'monetary', 'request', 'make', 'mention', 'try', 'start', 'argument', 'look', 'tonight', 'whatev', 'want', 'say', 'thread', 'enjoy', 'evening', 'write', 'must', 'read', 'opinion', 'column', 'put', 'together', 'investigation', 'mandate', 'incentive', 'impartial', 'agree', 'due', 'process', 'matter', 'far', 'survivor', 'get', 'inquiry', 'inquiry', 'firm', 'hire', 'second', 'people', 'say', 'investigate', 'nothing', 'find', 'incorrect', 'important', 'fact', 'employer', 'hire', 'firm', 'investigation', 'truly', 'independent', 'third', 'party', 'investigator', 'payment', 'come', 'seacrest', 'big', 'deal', 'finally', 'say', 'dare'

...

['nerd', 'specifically', 'working', 'pope', 'vatican', 'whatev', 'go', 'epstein', 'island', 'fully', 'aware', 'even', 'share', 'need', 'see', 'trust', 'rack', 'brain', 'night', 'always', 'assume', 'know', 'feel', 'whenever', 'qanon', 'post', 'sleep', 'last_night', 'matter', 'much', 'try', 'ignore', 'hence', 'continue', 'abramovic', 'exhausted', 'guy', 'keep', 'night', 'pyramid', 'evil', 'see', 'eye', 'cabal', 'whatev', 'fuck', 'want', 'call', 'power', 'vatican', 'necessarily', 'pope', 'current', 'evil', 'vatican', 'entity', 'alway', 'assume', 'eye', 'horu', 'speak', 'vatican', 'side', 'pyramid', 'soro', 'rothschild', 'flyrothsfly', 'course', 'interlink', 'royal', 'family', 'everywhere', 'get', 'start', 'actual', 'origin', 'blue', 'blood', 'believe', 'science', 'quick', 'note', 'everything', 'else', 'prob', 'stay', 'size', 'meter', 'high', 'accustomed', 'see', 'textbook', 'evolve', 'also', 'mean', 'shrink', 'size', 'else', 'survive', 'earth', 'strike', 'giant', 'humanoid', 'call', 'gian

['lucky', 'deal', 'loss', 'right', 'story', 'end', 'place', 'special', 'register', 'citizen', 'landlord', 'bank', 'employer', 'require', 'check', 'status', 'special', 'number', 'sum', 'look', 'large', 'group', 'people', 'people', 'contribute', 'country', 'many', 'decade', 'suddenly', 'ask', 'prove', 'status', 'environment', 'hostile', 'onus', 'person', 'investigation', 'citizen', 'want', 'rent', 'flat', 'person', 'also', 'want', 'citizen', 'citizen', 'give', 'landlord', 'special', 'number', 'check', 'special', 'register', 'prior', 'agreement', 'landlord', 'choose', 'else', 'equal', 'wide', 'context', 'course', 'strategy', 'fact', 'reach', 'continuously', 'extend', 'traced', 'tip', 'iceberg', 'evidence', 'overwhelming', 'lot', 'actively', 'create', 'next', 'windrush', 'generation', 'stop', 'never', 'forget', 'fond', 'human', 'right', 'act', 'implication', 'briton', 'government', 'want', 'replace', 'universal', 'human', 'right', 'act', 'authority', 'know', 'anything', 'much', 'status', '

['memo_nune', 'e', 'resume', 'fait', 'alternatifs', 'par', 'foxnew', 'alt', 'right', 'defense', 'trump', 'qui', 'sont', 'rendus', 'ici', 'honorable', 'mai', 'enjeu', 'est', 'pas', 'dan', 'partisan', 'trump', 'qui', 'sont', 'convaincus', 'la', 'conspiration', 'du', 'systeme', 'contre', 'lui', 'addendum', 'podcast', 'mon', 'passage', 'et', 'trump', 'dan', 'journal', 'mathieu', 'laurent', 'sur', 'addendum', 'durant', 'soiree', 'infos', 'sont', 'venus', 'discrediter', 'qui', 'pourrait', 'transformer', 'boomerang', 'pour', 'trump', 'part', 'sur', 'fox', 'meme', 'nune', 'admis', 'avoir', 'vu', 'lui', 'meme', 'demand', 'autorisation', 'ecoute', 'qu_il', 'critique', 'addendum', 'autre', 'part', 'break', 'du', 'cour', 'fisa', 'qui', 'delivra', 'autorisation', 'ecoute', 'sur', 'page', 'conseiller', 'trump', 'source', 'du', 'fbi', 'etait', 'enquete', 'politique', 'opposition', 'car', 'fbi', 'avait', 'informe', 'tout', 'savoir', 'si', 'opinion', 'majoritairement', 'accrochera', 'memo_nune', 'et', 

['robert_mercer', 'investor', 'study', 'university', 'illinois', 'ceo', 'renaissance', 'technology', 'old', 'hedge', 'fund', 'company', 'also', 'study', 'illinois', 'stock', 'mkt', 'shoot', 'suddenly', 'pull', 'investment', 'crash', 'indian', 'stock', 'market', 'plan', 'schedule', 'time', 'june', 'seem', 'postpone', 'october', 'december', 'guess', 'would', 'diwali', 'meet', 'other', 'india', 'visit', 'nix', 'well', 'connected', 'robert_mercer', 'investor', 'ca', 'scl', 'group', 'company', 'robert', 'know', 'well', 'political', 'circle', 'bomb', 'son', 'salil', 'pitroda', 'work', 'facebook', 'salil', 'son', 'organize', 'comedy', 'talk', 'show', 'world', 'work', 'hedge', 'fund', 'unit', 'citigroup', 'manhattan', 'hedge', 'fund', 'king', 'hedge', 'fund', 'robert_mercer', 'well', 'know', 'nix', 'seed', 'task', 'invest', 'indian', 'psu', 'big', 'company', 'shell', 'company', 'use', 'note', 'note', 'difficult', 'track', 'investment', 'size', 'cr', 'sir', 'explain', 'note', 'better', 'may', '

['fellow', 'healthcare', 'provider', 'remember', 'pain', 'important', 'vital', 'sign', 'always', 'document', 'qanon', 'background', 'info', 'bruce', 'nellie', 'orh', 'ham', 'radio', 'license', 'qanon', 'anon', 'make', 'observation', 'qanon', 'qanon', 'today', 'due', 'speculation', 'qanon', 'original', 'post', 'october', 'november', 'post', 'picture', 'seem', 'take', 'second', 'apart', 'leave', 'original', 'image', 'post', 'november', 'right', 'filename', '_', 'png', 'info', 'callaghan', 'trust', 'callaghan', 'qanon', 'qanon', 'ask', 'see', 'interview', 'paul', 'ryan', 'assume', 'speak', 'admit', 'president', 'trump', 'request', 'justice', 'department', 'look', 'politicization', 'warrant', 'future_prov', 'past', 'november', 'potus', 'visit', 'asia', 'qanon', 'post', 'image', 'note', 'filename', 'metadata', '_', 'png', 'kib', 'implication', 'travel', 'access', 'president', 'air', 'force', 'link', 'qanon', 'respond', 'speculation', 'qanon', 'compromise', 'subvert', 'deep_state', 'current'

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [14]:
for coro in corpus:
    print(coro)

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 3), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 6), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 3), (60, 1), (61, 1), (62, 2), (63, 1), (64, 2), (65, 3), (66, 2), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 6), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 2), (88, 1), (89, 2), (90, 1), (91, 1), (92, 2), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 4), (99, 1), (100, 1), (101, 1), (102, 1), (103, 2), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1),

[(54, 1), (63, 1), (81, 1), (85, 1), (92, 1), (98, 2), (115, 1), (136, 1), (137, 1), (141, 3), (146, 1), (157, 1), (163, 1), (180, 1), (237, 1), (242, 1), (243, 1), (246, 1), (264, 2), (265, 1), (270, 1), (279, 2), (296, 1), (326, 1), (338, 3), (376, 1), (388, 1), (409, 2), (428, 1), (437, 2), (438, 2), (452, 1), (456, 1), (471, 1), (493, 1), (531, 1), (561, 1), (577, 1), (579, 1), (659, 1), (660, 1), (661, 1), (662, 2), (663, 1), (664, 1), (665, 1), (666, 1), (667, 1), (668, 1), (669, 1), (670, 1), (671, 3), (672, 1), (673, 1), (674, 1), (675, 1), (676, 1), (677, 1), (678, 1), (679, 2), (680, 1), (681, 2), (682, 1), (683, 1), (684, 1), (685, 1), (686, 1), (687, 1), (688, 1), (689, 2), (690, 1), (691, 1), (692, 1), (693, 1), (694, 1), (695, 1), (696, 1), (697, 1), (698, 1), (699, 1), (700, 1), (701, 1), (702, 1), (703, 2), (704, 1), (705, 1), (706, 1), (707, 1), (708, 1), (709, 1), (710, 1), (711, 2), (712, 1), (713, 1), (714, 1), (715, 1), (716, 1), (717, 1), (718, 1), (719, 3), (720,

[(7, 1), (14, 1), (45, 1), (55, 2), (61, 5), (63, 2), (64, 1), (70, 1), (72, 3), (75, 1), (98, 1), (104, 1), (107, 2), (109, 1), (113, 3), (131, 1), (132, 2), (139, 1), (144, 1), (146, 1), (152, 1), (162, 3), (173, 1), (184, 1), (241, 1), (259, 2), (273, 3), (274, 2), (279, 3), (291, 2), (306, 1), (326, 2), (337, 1), (338, 1), (348, 1), (359, 1), (391, 1), (394, 2), (406, 1), (428, 2), (438, 1), (532, 2), (575, 1), (603, 1), (623, 1), (660, 1), (673, 2), (683, 1), (705, 1), (722, 1), (757, 1), (818, 1), (821, 1), (830, 2), (838, 1), (849, 1), (860, 2), (863, 1), (897, 2), (907, 1), (988, 1), (1002, 1), (1045, 2), (1067, 1), (1106, 1), (1107, 1), (1108, 3), (1109, 1), (1110, 1), (1111, 1), (1112, 1), (1113, 1), (1114, 1), (1115, 1), (1116, 1), (1117, 1), (1118, 1), (1119, 1), (1120, 1), (1121, 2), (1122, 1), (1123, 1), (1124, 2), (1125, 2), (1126, 1), (1127, 5), (1128, 1), (1129, 1), (1130, 3), (1131, 2), (1132, 1), (1133, 3), (1134, 1), (1135, 2), (1136, 2), (1137, 1), (1138, 1), (1139

[(61, 1), (62, 1), (80, 1), (97, 1), (98, 1), (103, 1), (115, 1), (117, 1), (121, 2), (139, 1), (146, 1), (152, 1), (165, 1), (173, 1), (174, 2), (183, 1), (184, 3), (195, 2), (220, 2), (244, 1), (259, 4), (265, 1), (274, 2), (299, 1), (313, 1), (326, 1), (333, 1), (336, 1), (348, 1), (356, 2), (359, 1), (360, 2), (364, 2), (378, 1), (382, 1), (391, 2), (398, 2), (412, 1), (438, 1), (445, 1), (447, 1), (452, 3), (453, 1), (510, 1), (531, 1), (540, 1), (545, 3), (549, 1), (595, 1), (618, 1), (624, 1), (656, 1), (666, 1), (747, 1), (760, 1), (764, 1), (794, 1), (832, 1), (837, 1), (839, 1), (843, 1), (867, 1), (874, 1), (890, 1), (921, 3), (987, 1), (992, 1), (1002, 1), (1036, 1), (1044, 2), (1050, 1), (1076, 1), (1107, 1), (1133, 1), (1146, 1), (1164, 1), (1187, 1), (1203, 1), (1219, 1), (1237, 2), (1303, 1), (1307, 1), (1311, 1), (1313, 1), (1349, 1), (1355, 1), (1452, 2), (1453, 1), (1454, 1), (1455, 1), (1456, 1), (1457, 1), (1458, 2), (1459, 1), (1460, 1), (1461, 1), (1462, 1), (146

[(41, 1), (61, 1), (62, 1), (70, 1), (85, 1), (104, 1), (133, 1), (138, 1), (161, 2), (162, 4), (165, 3), (183, 1), (184, 1), (185, 1), (259, 1), (273, 3), (274, 1), (284, 2), (310, 2), (314, 1), (332, 1), (337, 1), (348, 3), (350, 1), (355, 1), (356, 2), (358, 1), (359, 1), (371, 1), (378, 1), (391, 1), (435, 1), (446, 1), (531, 1), (546, 1), (569, 1), (583, 1), (584, 1), (596, 1), (602, 1), (631, 1), (635, 1), (644, 2), (657, 1), (661, 2), (778, 1), (781, 1), (805, 1), (871, 1), (879, 1), (880, 1), (883, 1), (890, 1), (913, 1), (921, 1), (997, 1), (1037, 2), (1038, 2), (1210, 1), (1225, 3), (1237, 2), (1259, 1), (1286, 1), (1307, 2), (1344, 1), (1346, 1), (1403, 1), (1463, 1), (1494, 1), (1503, 1), (1724, 1), (1839, 1), (1861, 1), (1862, 1), (1863, 1), (1864, 1), (1865, 1), (1866, 2), (1867, 1), (1868, 1), (1869, 1), (1870, 1), (1871, 1), (1872, 1), (1873, 1), (1874, 1), (1875, 1), (1876, 1), (1877, 1), (1878, 1), (1879, 1), (1880, 1), (1881, 2), (1882, 2), (1883, 1), (1884, 1), (188

...

[(7, 1), (14, 3), (61, 2), (62, 1), (63, 6), (64, 1), (70, 1), (81, 1), (85, 1), (94, 3), (98, 2), (107, 1), (109, 2), (131, 1), (133, 1), (136, 1), (144, 1), (146, 5), (149, 1), (162, 1), (164, 2), (171, 2), (173, 1), (174, 1), (175, 1), (178, 1), (180, 2), (195, 1), (197, 1), (198, 1), (207, 1), (220, 1), (221, 1), (233, 1), (237, 1), (243, 2), (244, 1), (246, 3), (259, 7), (261, 1), (270, 1), (273, 3), (278, 2), (279, 2), (284, 2), (301, 1), (310, 1), (313, 1), (316, 1), (317, 3), (326, 1), (327, 1), (342, 1), (348, 1), (356, 4), (359, 1), (371, 1), (373, 1), (378, 3), (380, 1), (382, 1), (384, 1), (394, 2), (421, 1), (428, 37), (447, 1), (452, 6), (453, 1), (455, 1), (463, 1), (465, 1), (471, 1), (485, 1), (510, 2), (511, 1), (549, 4), (556, 3), (585, 2), (587, 2), (605, 1), (618, 2), (636, 2), (660, 1), (666, 1), (683, 5), (722, 1), (726, 1), (730, 1), (778, 1), (781, 1), (797, 1), (808, 2), (809, 1), (832, 1), (848, 2), (849, 1), (850, 1), (853, 8), (894, 1), (914, 1), (918, 1), 

[(3, 3), (26, 1), (61, 1), (62, 1), (97, 2), (98, 1), (107, 11), (121, 1), (122, 1), (136, 3), (152, 1), (153, 3), (164, 1), (168, 2), (170, 2), (175, 1), (182, 1), (189, 1), (190, 29), (195, 2), (221, 1), (237, 1), (243, 1), (270, 1), (273, 5), (274, 6), (337, 1), (348, 1), (387, 2), (395, 1), (406, 1), (418, 1), (423, 1), (428, 27), (435, 1), (438, 2), (456, 1), (467, 10), (470, 1), (491, 9), (492, 1), (496, 1), (501, 3), (505, 2), (506, 7), (510, 1), (544, 6), (563, 1), (571, 1), (576, 3), (584, 1), (595, 1), (599, 1), (603, 1), (611, 1), (625, 1), (631, 1), (635, 1), (683, 1), (719, 1), (723, 1), (736, 1), (738, 1), (741, 1), (747, 3), (749, 1), (757, 1), (764, 3), (768, 1), (785, 1), (798, 1), (802, 1), (821, 1), (830, 1), (835, 1), (838, 1), (860, 1), (862, 1), (863, 1), (889, 2), (890, 1), (914, 1), (919, 2), (925, 1), (931, 1), (952, 1), (954, 1), (957, 1), (965, 10), (979, 1), (982, 1), (1014, 1), (1028, 1), (1036, 1), (1038, 1), (1044, 1), (1127, 1), (1137, 1), (1162, 1), (11

[(25, 1), (55, 1), (64, 1), (81, 1), (104, 1), (110, 1), (115, 2), (131, 1), (141, 1), (144, 1), (162, 1), (169, 1), (173, 2), (183, 1), (186, 2), (190, 22), (212, 1), (221, 1), (227, 1), (259, 3), (274, 1), (317, 1), (356, 1), (373, 1), (401, 1), (402, 1), (403, 1), (428, 2), (431, 1), (465, 1), (467, 5), (491, 6), (496, 1), (510, 1), (518, 1), (535, 1), (540, 1), (563, 1), (579, 1), (584, 1), (611, 1), (652, 2), (683, 1), (693, 1), (708, 1), (722, 1), (738, 1), (772, 1), (778, 1), (785, 1), (801, 1), (810, 1), (818, 1), (832, 1), (860, 1), (874, 2), (897, 1), (921, 2), (952, 2), (959, 1), (977, 1), (1002, 1), (1044, 1), (1069, 1), (1089, 1), (1105, 1), (1135, 1), (1155, 2), (1174, 1), (1265, 1), (1270, 1), (1303, 1), (1309, 1), (1318, 1), (1339, 4), (1342, 1), (1351, 3), (1470, 1), (1490, 1), (1503, 1), (1524, 1), (1621, 1), (1797, 2), (1850, 1), (1879, 1), (1886, 2), (1904, 1), (1931, 1), (1947, 1), (2131, 1), (2172, 1), (2219, 2), (2224, 1), (2243, 1), (2535, 1), (2597, 1), (2642, 

[(10, 2), (41, 1), (42, 2), (113, 1), (157, 1), (189, 2), (302, 1), (333, 1), (337, 1), (398, 1), (409, 5), (428, 27), (434, 1), (463, 5), (471, 1), (472, 1), (585, 5), (599, 6), (625, 2), (683, 14), (760, 2), (932, 1), (1034, 4), (1059, 1), (1151, 2), (1174, 1), (1203, 1), (1311, 3), (1337, 2), (1352, 9), (1360, 2), (1364, 4), (1454, 1), (1464, 1), (1500, 1), (1515, 1), (1521, 1), (1529, 8), (1530, 3), (1537, 4), (1539, 2), (1540, 1), (1541, 2), (1546, 2), (1549, 5), (1557, 9), (1564, 1), (1566, 14), (1567, 2), (1571, 1), (1572, 1), (1578, 3), (1579, 12), (1580, 5), (1583, 2), (1600, 1), (1607, 14), (1614, 2), (1615, 1), (1621, 3), (1627, 2), (1630, 9), (1633, 2), (1635, 1), (1637, 13), (1639, 1), (1644, 2), (1646, 1), (1647, 10), (1655, 2), (1660, 2), (1661, 5), (1662, 4), (1671, 1), (1680, 1), (1682, 1), (1683, 1), (1687, 1), (1715, 1), (1772, 3), (1862, 5), (1943, 3), (1948, 2), (2083, 1), (2178, 4), (2233, 2), (2336, 1), (2463, 5), (2527, 1), (2701, 1), (2774, 1), (2927, 1), (2980

[(14, 1), (61, 2), (62, 1), (63, 4), (94, 1), (98, 1), (107, 1), (113, 1), (132, 1), (144, 1), (147, 1), (172, 1), (174, 1), (183, 2), (233, 1), (241, 1), (243, 1), (246, 1), (259, 2), (270, 2), (273, 2), (274, 2), (283, 1), (284, 1), (310, 1), (311, 1), (332, 1), (378, 2), (399, 1), (458, 1), (470, 1), (493, 1), (529, 1), (549, 1), (559, 1), (576, 2), (588, 1), (607, 1), (673, 1), (676, 1), (736, 1), (757, 1), (820, 1), (821, 1), (830, 1), (874, 1), (896, 1), (928, 1), (933, 1), (934, 1), (993, 2), (997, 1), (1004, 1), (1037, 1), (1090, 1), (1105, 1), (1107, 1), (1140, 3), (1183, 1), (1196, 1), (1235, 1), (1237, 1), (1244, 1), (1286, 1), (1309, 1), (1351, 1), (1353, 2), (1380, 1), (1395, 1), (1401, 1), (1607, 1), (1674, 2), (1766, 1), (1868, 1), (1998, 1), (2124, 5), (2131, 1), (2145, 1), (2175, 1), (2363, 1), (2376, 1), (2381, 1), (2400, 1), (2408, 1), (2494, 1), (2558, 1), (2576, 1), (2582, 1), (2600, 5), (2738, 2), (2791, 1), (2895, 2), (2946, 1), (2960, 1), (3285, 2), (3320, 1), (

In [15]:
THIS_FOLDER = os.getcwd()
carpeta_mallet = "mallet-2.0.8"
bin_c = "bin"
archivo = "mallet"
camino = os.path.join(THIS_FOLDER, carpeta_mallet)
mallet_path_1 = os.path.join(camino, bin_c)
mallet_path = os.path.join(mallet_path_1,archivo)


os.environ['MALLET_HOME'] = camino

In [16]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [17]:
inicio = 2
limite = 60
pasos = 1

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=inicio, limit=limite, step=pasos)

In [ ]:
# Show graph
limit=limite
start=inicio
step=pasos;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[8]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=20))